In [1]:
import numpy as np
from IPython.display import clear_output

In [2]:
def str_jogo(jogo):
    """
    Transforma uma string de visualização do jogo no array 'readable' para o script.
    """
    a = 0
    jogo_final = np.zeros([6,7], dtype=str) #criação do array
    #l = np.zeros([7], dtype=str)
    cl=0
    for i in jogo:
        if i == "\n": 
            continue #ignora as o caracter de mudança de linha
        jogo_final[cl,a] = i
        a=a+1
        if a ==7:
            a=0
            cl=cl+1  #começa uma linha nova
    return jogo_final


In [3]:
def jogo_str(jogo):
    """
    Transforma o jogo no array 'readable' para o script numa string de visualização.
    """
    s = ""
    for a in jogo:
        for b in a:
            s=s+b
        s=s+"\n"
    return s

In [4]:
def ganho(state):
    """
    Recebe um estado e retorna o vencedor (no caso deste existir)
    
    """

    #Itera sobre todos os segmentos de tamanho 4 e verifica se há condição de vencedor
    for segment in get_segments(state):
        if np.array_equal(segment, ['X', 'X', 'X', 'X']):
            return "X"
        elif np.array_equal(segment, ['O', 'O', 'O', 'O']):
            return "O"
    return None
        

In [5]:
def all_actions(state):
    """
    Recebe um estado e retorna uma lista com todas as ações possíveis.

    """
    actions = []

    #Para cada coluna verifica se a primeira posição (posição mais superior do tabuleiro) 
    #está livre.

    for i in range(7):
        if state[0][i] == "-":
            actions.append(i)
    if len(actions)==0:
        return None
    return actions

In [6]:
def player(state):
    """
    Recebe um estado e retorna o jogador nesse turno.

    """


    cx = 0 #contador de X
    co = 0 #contador de O
    for i in range(6):
        a = state[i]
        for j in range(7):
            b = a[j]
            if b == "X":
                cx +=1
            elif b == "O":
                co +=1
    #caso do tabuleiro estar completamente ocupado
    if cx+co == 7*6:
        return None
    
    #Se o número de X's for menor ou igual ao numéro de O's, então é a vez de X jogar.
    if cx <=co:
        return "X"
    else:
        return "O"

In [7]:
def result(jogo, inp):
    """
    Recebe o (estado do) jogo e a ação (coluna onde quer colocar a peça). Coloca a peça no espaço mais abaixo disponível na coluna indicada.
    Se a jogada não for válida (coluna inexistente ou inocupada) retorna None.

    """

    #Verifica se a coluna de input é inexistente, retornando None nesse caso
    if inp>6 or inp<0:
        return None
    pl = player(jogo)
    jogo_f = -1
    col_u = jogo[:, inp] #Seleciona a coluna
    for ind, i in enumerate(col_u[::-1]):  #Enumera a coluna selecionada começando do fim para o início
        if i == "-": #Encontra o primeiro espaço vazio
            jogo_f = np.copy(jogo) #Copia o (estado do) jogo 
            jogo_f[5-ind,inp] = pl #Coloca a peça do jogador no local apropriado
            return jogo_f
    if jogo_f==-1:
        raise Exception("Impossible Move")
    return None

In [8]:
def terminal(state):
    """"
    Verifica se um determinado estado é um estado terminal/final 
     (estado em que um dos jogadores ganhou ou em que não há ações possíveis)
    
    """

    #Se houver vencedor o jogo acaba
    if ganho(state):
        return True
    
    #Se não houverem ações disponiveis o jogo acaba
    elif len(all_actions(state))==0:
        return True
    else:
        return False

In [9]:
def utility(state):
    """
    Recebe um estado e retorna a utilidade desse estado
      (512 para vitória de X, -512 para vitória de O e 0 em caso de empate).
    
    """
    win = ganho(state)
    if win =="X":
        return 512
    elif win =="O":
        return -512
    else:
        return 0

In [10]:
def get_segments(state):
    """
    Recebe um estado e retorna uma lista com todos os segmentos de tamanho 4 existentes.
    
    """
    segments = [] 

    #Verifica as linhas e colunas
    for i in range(6):
        linha = state[i]
        for j in range(4):
            segments.append(linha[j:j+4])
        coluna = state[:,i]
        for j in range(3):
            segments.append(coluna[j:j+4])

    #Verifica a ultima coluna       
    coluna = state[:,6]
    for j in range(3):
        segments.append(coluna[j:j+4])
    
    #Verifica as diagonais principais
    for i in range(-2,4):
        dia = np.diag(state,i)
        for j in range(len(dia)-3):
            segments.append(dia[j:j+4])
    #Dá flip no array e verifica as diagonais principais do array flipado (equivalentes às diagonais perpendiculares às principais do array original)
    state_tr = np.fliplr(state)
    for i in range(-2,4):
        dia = np.diag(state_tr,i)
        for j in range(len(dia)-3):
            segments.append(dia[j:j+4])
        
    
    return segments

In [11]:
def evaluate(segment):
    cx = 0
    co = 0
    for i in segment:
        if i=="X":
            cx+=1
        elif i=="O":
            co+=1

   # print(cx, co)
    if (cx==0 and co==0) or (cx>0 and co>0):
        return 0
    elif cx == 1:
        return 1
    elif cx == 2:
        return 10
    elif cx==3:
        return 50
    
    #   VERIFICAR ISTO
    elif cx == 4:
        return 512
    elif co ==4:
        return -512
    # ---

    elif co == 1:
        return -1
    elif co == 2:
        return -10
    elif co==3:
        return -50

In [12]:
def evaluate_all(state):

    win = ganho(state)
    if win =="X":
        return 512
    elif win =="O":
        return -512
    elif terminal(state):
        return 0
    s = 0
    if player(state)=="X":
        s= s+16
    elif player(state)=="O":
        s=s-16
    for segment in get_segments(state):
        s=s+evaluate(segment)
    return s

In [13]:
def alphabeta(state, depth, alpha, beta):

    """
    Aplica o algoritmo min/max aproximado (como descrito no enunciado).
    Aplica também alpha-betta pruning.
    Recebe um estado de jogo, um limite máximo de profundiade e alpha beta.
    Utiliza alpha-beta até à profundidade limite retornando uma aproximação do valor real.
    Retorna o valor min/max e a ação correspondente.
    Avalia automaticamente se é o max player ("X") ou min player ("O").
    
    """
    #print(state)
    if depth ==0:
        segments = get_segments(state)
        s = 0
        for segment in segments:
            #print(segment)
            s = s + evaluate(segment)

        #PARTE NOVA QUE ADICIONEI
        pl = player(state)
        if pl =="X":
            s =s+16
        elif pl=="O":
            s=s-16
        # -------------
        return s, None
    
    #FAZER COM QUE RETORNE O MOVIMENTO QUE FAZ GANHAR
    if terminal(state):
        return utility(state), None
    
    if player(state) == "X":
        v = -np.infty
        move = None
        for action in all_actions(state):
            res = result(state, action)
            test = alphabeta(res, depth-1, alpha, beta )[0]
            #print(test)
            if test >v:
                v = test 
                move = action
            if v > beta:
                break
            alpha = max(alpha, v)
        return v, move
    else:
        v = np.infty
        for action in all_actions(state):
            res = result(state, action)
            test = alphabeta(res, depth-1, alpha, beta )[0]
            if test < v:
                v = test 
                move = action
            if v < alpha:
                break
            beta = min(beta, v)        
        return v, move

In [14]:
# É a equivalente à alpha-beta só que sem o alphabeta pruning. É Só minimax

def minimax(state, depth=3):
    if depth == 0:
        return evaluate_all(state), None

    if terminal(state):
        return utility(state), None
    
    if player(state) == "X":
        v = -np.infty
        move = None
        for action in all_actions(state):
            res = result(state, action)
            test = minimax(res, depth - 1)[0]
            if test > v:
                v = test 
                move = action
        return v, move
    else:
        v = np.infty
        move = None
        for action in all_actions(state):
            res = result(state, action)
            test = minimax(res, depth - 1)[0]
            if test < v:
                v = test 
                move = action
        return v, move


# Testes - Alphabeta

In [15]:
ji = """-------
-------
-------
-------
-------
-------"""

ji = str_jogo(ji)

#print(alphabeta(ji,5, -np.infty, np.infty))

print(ji)

[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']]


In [31]:
ji = result(ji, alphabeta(ji,5,-np.infty, np.infty)[1])

print(ji, ganho(ji))
if ganho(ji):
    print("JA GANHARAM:", ganho(ji))

[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' 'X' '-' '-' '-' '-']
 ['-' '-' 'O' '-' '-' '-' '-']
 ['-' '-' 'X' '-' '-' '-' '-']] None


In [32]:
ji = result(ji, 2)
print(ji)
if ganho(ji):
    print("JA GANHARAM:", ganho(ji))

[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' 'O' '-' '-' '-' '-']
 ['-' '-' 'X' '-' '-' '-' '-']
 ['-' '-' 'O' '-' '-' '-' '-']
 ['-' '-' 'X' '-' '-' '-' '-']]


# Teste Minimax

In [16]:
#Iniciar jogo

ji = """-------
-------
-------
-------
-------
-------"""

ji = str_jogo(ji)

#print(alphabeta(ji,5, -np.infty, np.infty))

print(ji)

[['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']
 ['-' '-' '-' '-' '-' '-' '-']]


In [41]:
# Vez do jogador 

ji = result(ji, 6)
print(ji)
if ganho(ji):
    print("JA GANHARAM:", ganho(ji))

[['O' '-' '-' '-' '-' '-' '-']
 ['X' '-' 'O' '-' '-' '-' '-']
 ['O' 'O' 'X' 'O' '-' '-' '-']
 ['O' 'X' 'O' 'O' 'O' '-' '-']
 ['O' 'X' 'O' 'X' 'X' '-' 'X']
 ['X' 'X' 'X' 'O' 'X' 'X' 'X']]


In [42]:
#Vez da IA

ji = result(ji, minimax(ji)[1])

print(ji, ganho(ji))
if ganho(ji):
    print("JA GANHARAM:", ganho(ji))

[['O' '-' '-' '-' '-' '-' '-']
 ['X' '-' 'O' '-' '-' '-' '-']
 ['O' 'O' 'X' 'O' '-' '-' '-']
 ['O' 'X' 'O' 'O' 'O' '-' '-']
 ['O' 'X' 'O' 'X' 'X' 'O' 'X']
 ['X' 'X' 'X' 'O' 'X' 'X' 'X']] O
JA GANHARAM: O


# TESTE DE JOGO

In [27]:
ji = """-------
-------
-------
-------
-------
-------"""

ji = str_jogo(ji)

vn = ganho(ji)

while terminal(ji)==False:
    clear_output()
    ji = result(ji, alphabeta(ji,5,-np.infty, np.infty)[1])

    print(ji, ganho(ji))
    inp = int(input())
    ji = result(ji, inp)
    print(ji)

print(ji, ganho(ji))
print("GANHOU:", ganho(ji))


[['-' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' '-']
 ['X' '-' '-' '-' '-' '-' '-']
 ['O' '-' '-' '-' '-' '-' '-']
 ['O' '-' 'X' '-' '-' '-' '-']
 ['O' '-' 'X' 'X' 'X' '-' '-']] None


ValueError: invalid literal for int() with base 10: ''

In [63]:


jogo = """-------
-------
-------
-------
OOOO-XO
X-OXOXO"""

jogo = str_jogo(jogo)
print("1-> Caso de 4 numa linha")
print(jogo_str(jogo))
print("---")


jogo2 = """-------
-------
X------
X------
X--O-XO
X-OXOXO"""

print("2-> Caso de 4 numa coluna")
print(jogo_str(str_jogo(jogo2)))
jogo2 = str_jogo(jogo2)
print("---")





jogo3 = """-------
-------
---X---
--X----
-X---XO
X-OXOXO"""

jogo3 = str_jogo(jogo3)
print("3-> Caso de uma diagonal não principal")
print(jogo_str(jogo3))
print("---")


jogo4 = """X------
X------
X------
X------
X--O-XO
X-OXOXO"""

print("4-> Caso de uma coluna cheia")
print(jogo_str(str_jogo(jogo4)))
jogo4 = str_jogo(jogo4)
print("---")


1-> Caso de 4 numa linha
-------
-------
-------
-------
OOOO-XO
X-OXOXO

---
2-> Caso de 4 numa coluna
-------
-------
X------
X------
X--O-XO
X-OXOXO

---
3-> Caso de uma diagonal não principal
-------
-------
---X---
--X----
-X---XO
X-OXOXO

---
4-> Caso de uma coluna cheia
X------
X------
X------
X------
X--O-XO
X-OXOXO

---


In [35]:
all_actions(jogo4)

[1, 2, 3, 4, 5, 6]

In [36]:
print(result(jogo4, 1))


[['X' '-' '-' '-' '-' '-' '-']
 ['X' '-' '-' '-' '-' '-' '-']
 ['X' '-' '-' '-' '-' '-' '-']
 ['X' '-' '-' '-' '-' '-' '-']
 ['X' '-' '-' 'O' '-' 'X' 'O']
 ['X' 'X' 'O' 'X' 'O' 'X' 'O']]


In [74]:
ganho(jogo4)

'X'

In [49]:
print(jogo_str(jogo4))

X------
X------
X------
X------
X--O-XO
X-OXOXO



In [72]:
for i in get_segments(jogo4):
    if np.array_equal(i,["X", "-", "-", "-"]):
        print(i)


['X' '-' '-' '-']
['X' '-' '-' '-']
['X' '-' '-' '-']
['X' '-' '-' '-']
['X' '-' '-' '-']


In [66]:
print(ganho(jogo3))

X


In [67]:
jf = np.fliplr(jogo)
np.diag(jogo, -2)

array(['-', '-', 'O', 'X'], dtype='<U1')

In [97]:
print(evaluate(["-", "-", "-", "-"]))




0 0
0


In [26]:
vx = 0
vo = 0
em = 0
sf =[]
for i in range(3):
    ji = """-------
-------
-------
-------
-------
-------"""
    state = str_jogo(ji)
    #print(terminal(state))
    while terminal(state) == False:
        state = result(state, alphabeta(state, 5, -np.infty, np.infty)[1])
        #print(state)
    if ganho(state) == "X":
        vx = vx+1
    elif ganho(state) == "O":
        vo = vo +1
    else:
        em =em+1
    print(i)
    sf.append(state)
print(vx, vo, em)

0
1
2
3 0 0


In [29]:
for i in sf:
    print(i)
    print("........................")

[['O' 'O' 'X' '-' 'X' '-' '-']
 ['X' 'X' 'O' '-' 'O' 'X' '-']
 ['O' 'O' 'X' 'X' 'X' 'X' '-']
 ['O' 'X' 'X' 'O' 'O' 'O' '-']
 ['X' 'O' 'O' 'X' 'X' 'O' '-']
 ['X' 'O' 'X' 'O' 'O' 'X' '-']]
........................
[['O' 'O' 'X' '-' 'X' '-' '-']
 ['X' 'X' 'O' '-' 'O' 'X' '-']
 ['O' 'O' 'X' 'X' 'X' 'X' '-']
 ['O' 'X' 'X' 'O' 'O' 'O' '-']
 ['X' 'O' 'O' 'X' 'X' 'O' '-']
 ['X' 'O' 'X' 'O' 'O' 'X' '-']]
........................
[['O' 'O' 'X' '-' 'X' '-' '-']
 ['X' 'X' 'O' '-' 'O' 'X' '-']
 ['O' 'O' 'X' 'X' 'X' 'X' '-']
 ['O' 'X' 'X' 'O' 'O' 'O' '-']
 ['X' 'O' 'O' 'X' 'X' 'O' '-']
 ['X' 'O' 'X' 'O' 'O' 'X' '-']]
........................


# Testes de tempo de execução

In [37]:
%timeit ganho(jogo)


260 µs ± 6.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# TO-DO
 
* Verificar se um dado estado de jogo é possível
* Criar algoritmo A-star
* Criar MCTS
* Criar o jogo-teste
* Criar ciclo de jogo
* Avaliar tempos de execução